### 1.0 -Load libraries
In our Second Hand Car Price Prediction project we used SXBoost, mada score

In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # library for visualization
import seaborn as sns # library for visualization
import warnings
import os
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

warnings.filterwarnings('ignore')
sns.set_palette('Spectral')
sns.set_context('notebook', font_scale=1)
sns.set_style('whitegrid')
# set pandas 3 digits decimal
pd.set_option('display.float_format', '{:.3f}'.format)

<h4> 1.1 Load the available data </h4>

In [51]:
dir = 'vehicles-random-subset.csv'
with open(dir, mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None) 
    
# print first 5 recording of data
df=pd.read_csv('vehicles-random-subset.csv')
df.columns = df.columns.str.lower()  # Convert all column names to lowercase
print(df.head(5))
print()
print(df.columns)


  year;manufacturer;model;condition;cylinders;fuel;odometer;title_status;transmission;drive;size;type;paint_color;price
0  1950;ford;m-deluxe;excellent;'6 cylinders';gas...                                                                   
1  1950;chevrolet;'m-3100 truck';good;'6 cylinder...                                                                   
2  1950;ford;m-sedan;good;'8 cylinders';gas;47133...                                                                   
3  1950;chevrolet;'m-sport coupe';excellent;'8 cy...                                                                   
4  1950;chevrolet;'m-sport coupe';'like new';'8 c...                                                                   

Index(['year;manufacturer;model;condition;cylinders;fuel;odometer;title_status;transmission;drive;size;type;paint_color;price'], dtype='object')


#### 1.2 Scrape data from internet and append the data to available data

This is done to increase amount of data and to increase coherence.

In [52]:
#get further data, clean it and then append to the vehicles-random-subset.csv
# ATTRIBUTES ARE:
#['year','manufacturer','model','condition','cylinders','fuel','odometer','title_status','transmission','drive','size','type','paint_color','price']

### 2.0 Train/Test splitting

All the collected data needs to be split to test, train. We will split 20/80 = test/train

In [53]:
df.columns = df.columns.str.strip().str.lower()
df = pd.read_csv('vehicles-random-subset.csv', delimiter=';')

categorical_columns = df.columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()  # Create an instance of LabelEncoder
    df[col] = le.fit_transform(df[col].astype(str))  # Fit and transform the column
    label_encoders[col] = le

#features (X) and target (y)
X = df.drop(columns='price')
y = df['price']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### XGBoost regressor

In [54]:
import xgboost as xgb
# Train an XGBoost regressor model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### MAP

In [55]:
import numpy as np
import random

def apk(actual, predicted, k=10):
    """
    Average Precision at K
    :param actual: Gerçek etiketler (list)
    :param predicted: Tahmin edilen etiketler (list)
    :param k: İlk k tahmini değerlendirmek için (default=10)
    :return: Ortalama hassasiyet (float)
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

def mapk(actual_list, predicted_list, k=10):
    """
    Mean Average Precision at K
    :param actual_list: Tüm gerçek etiketler (list of lists)
    :param predicted_list: Tüm tahmin edilen etiketler (list of lists)
    :param k: İlk k tahmini değerlendirmek için (default=10)
    :return: MAP@K skoru (float)
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual_list, predicted_list)])

# Tahminleri bozarak MAP@3 oranını azaltma
def reduce_map_predictions(predictions, noise_ratio=0.5, num_labels=10):
    """
    Tahminleri bozarak MAP oranını düşürür.
    :param predictions: Tahmin edilen etiketler (list of lists)
    :param noise_ratio: Gürültü oranı (0-1 arası, doğru tahminleri bozma oranı)
    :param num_labels: Rastgele seçilecek etiketlerin toplam sayısı
    :return: Gürültü eklenmiş tahminler
    """
    noisy_predictions = []
    for pred in predictions:
        noise_count = int(len(pred) * noise_ratio)
        noise = [random.randint(1, num_labels) for _ in range(noise_count)]
        noisy_predictions.append(pred + noise)  # Gürültü ekle
        random.shuffle(noisy_predictions[-1])  # Sıralamayı karıştır
    return noisy_predictions

# Örnek veri
actual = [[1, 2, 3], [1, 2, 3], [1, 2]]
predicted = [[1, 2, 3], [2, 3, 4], [1, 3, 5]]

# Tahminlere gürültü ekleme
noisy_predicted = reduce_map_predictions(predicted, noise_ratio=0.7, num_labels=10)

# MAP@3 Hesaplama
map_score = mapk(actual, noisy_predicted, k=3)
print(f"MAP@3 (Gürültü eklenmiş): {map_score}")


MAP@3 (Gürültü eklenmiş): 0.16666666666666666


In [56]:
import numpy as np
import random

def apk(actual, predicted, k=10):
    """
    Average Precision at K
    :param actual: Ground truth (list)
    :param predicted: Predicted labels (list)
    :param k: Evaluate the first k predictions (default=10)
    :return: Average precision (float)
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

def mapk(actual_list, predicted_list, k=10):
    """
    Mean Average Precision at K
    :param actual_list: List of ground truth values (list of lists)
    :param predicted_list: List of predicted labels (list of lists)
    :param k: Evaluate the first k predictions (default=10)
    :return: MAP@K score (float)
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual_list, predicted_list)])

def precision_recall(actual, predicted, k=10):
    """
    Precision and Recall at K
    :param actual: Ground truth (list)
    :param predicted: Predicted labels (list)
    :param k: Evaluate the first k predictions (default=10)
    :return: Precision and Recall values (tuple)
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    # Calculate true positives
    true_positives = len([p for p in predicted if p in actual])
    false_positives = len([p for p in predicted if p not in actual])
    false_negatives = len([a for a in actual if a not in predicted])

    # Precision and Recall calculations
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0

    return precision, recall

# Reduce MAP predictions to add noise
def reduce_map_predictions(predictions, noise_ratio=0.5, num_labels=10):
    """
    Add noise to predictions to reduce MAP@K.
    :param predictions: Predicted labels (list of lists)
    :param noise_ratio: Noise ratio (0-1, proportion of predictions to corrupt)
    :param num_labels: Total number of labels to randomly sample from
    :return: Noisy predictions
    """
    noisy_predictions = []
    for pred in predictions:
        noise_count = int(len(pred) * noise_ratio)
        noise = [random.randint(1, num_labels) for _ in range(noise_count)]
        noisy_predictions.append(pred + noise)  # Add noise
        random.shuffle(noisy_predictions[-1])  # Shuffle
    return noisy_predictions

# Example data
actual = [[1, 2, 3], [1, 2, 3], [1, 2]]
predicted = [[1, 2, 3], [2, 3, 4], [1, 3, 5]]

# Add noise to predictions
noisy_predicted = reduce_map_predictions(predicted, noise_ratio=0.7, num_labels=10)

# MAP@3 calculation
map_score = mapk(actual, noisy_predicted, k=3)
print(f"MAP@3 (Noisy): {map_score}")

# Precision and Recall for each instance
for i, (a, p) in enumerate(zip(actual, noisy_predicted)):
    precision, recall = precision_recall(a, p, k=3)
    print(f"Instance {i + 1}: Precision@3 = {precision:.2f}, Recall@3 = {recall:.2f}")


MAP@3 (Noisy): 0.4259259259259259
Instance 1: Precision@3 = 1.00, Recall@3 = 0.75
Instance 2: Precision@3 = 0.33, Recall@3 = 0.33
Instance 3: Precision@3 = 0.33, Recall@3 = 0.50
